In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import random
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from modelling import *
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping
import pickle
import random
import string
from sklearn.preprocessing import LabelEncoder

In [ ]:
%load_ext autoreload
%autoreload 2
from modelling import *

In [ ]:
data_class  = DataClass(use_prediction= True,use_enhanced="lstm",custom=True)
data_class.create_dataset()
modelling_data = DataModel(data = data_class.data_model,
                           use_enhanced= data_class.use_enhanced,
                           custom_test_index=data_class.enhanced_index,
                           max_len=200,
                           custom=True)
modelling_data.create_padding()
modelling_data.test

In [ ]:
freq_name = data_class.freq_name.copy()
freq_name = freq_name[freq_name['total']>1000].sort_values(by = 'total')
data_model_translate = freq_name[["firstname"]]
data_model_translate["Y"] = data_model_translate['firstname']
data_model_translate.columns = ["X","Y"]
data_model_translate

In [ ]:

def alter_word(word):
    operations = []
    if len(word) < 3:
        possible_ops = ['replace_one', 'delete_one']
    else:
        possible_ops = ['replace_two', 'replace_one', 'delete_one', 'delete_two']

    for op in possible_ops:
        if op == 'replace_one':
            pos = random.randint(0, len(word) - 1)
            new_letter = random.choice(string.ascii_letters)
            new_word = word[:pos] + new_letter + word[pos + 1:]
            new_word = str.lower(new_word)
            operations.append(new_word)

        elif op == 'replace_two':
            pos1, pos2 = random.sample(range(len(word)), 2)
            new_letter1, new_letter2 = random.sample(string.ascii_letters, 2)
            new_word = list(word)
            new_word[pos1], new_word[pos2] = new_letter1, new_letter2
            
            operations.append(str.lower(''.join(new_word)))

        elif op == 'delete_one':
            pos = random.randint(0, len(word) - 1)
            new_word = word[:pos] + word[pos + 1:]
            new_word = str.lower(new_word)
            operations.append(new_word)

        elif op == 'delete_two':
            pos1, pos2 = random.sample(range(len(word)), 2)
            new_word = ''.join(letter for i, letter in enumerate(word) if i not in [pos1, pos2])
            new_word = str.lower(new_word)
            operations.append(new_word)

    return operations

data = []
for ope in tqdm(range(50)):
    for index in range(data_model_translate.shape[0]):
        name,true =data_model_translate.iloc[index]
        altered_name = alter_word(name)   
        out = pd.DataFrame([altered_name,[true]*len(altered_name)]).T
        data.append(out)
    
data_model = pd.concat(data,axis=0)
data_model.columns = ['X','Y']

In [ ]:

label_encoder = LabelEncoder()
# Tokenize words
tokenizer = Tokenizer(num_words=500, oov_token="<OOV>",char_level=True)
tokenizer.fit_on_texts(data_model['X'])
X_seq = tokenizer.texts_to_sequences(data_model['X'])
X_padded = pad_sequences(X_seq, padding='post', maxlen=8)

Y_encoded = label_encoder.fit_transform(data_model['Y'])
Y_categorical = to_categorical(Y_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X_padded, Y_categorical, test_size=0.1, random_state=42)
# Assuming your LabelEncoder is named label_encoder and has been fitted

# Saving the LabelEncoder
# with open('label_encoder.pickle', 'wb') as handle:
#     pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)


LSTM

In [ ]:
model = Sequential([
    Embedding(500, 16, input_length=8),
    Bidirectional(LSTM(100, return_sequences=True)),
    Bidirectional(LSTM(100)),
    Dense(100, activation='relu'),
    Dense(Y_categorical.shape[1], activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model.summary()


early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(X_train,
                    Y_train,
                    epochs=20,
                    validation_data=(X_test, Y_test),
                    validation_split = 0.25,
                    callbacks = [early_stopping],
                    verbose=2)

In [ ]:
model.save('my_model.h5')
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# Loading the LabelEncoder
with open('label_encoder.pickle', 'rb') as handle:
    loaded_label_encoder = pickle.load(handle)


In [ ]:
# First, we tokenize and pad the new X_test strings as we did with the training data
X_test_new =data_class.data.loc[data_class.enhanced_index]['firstname'].str.lower().values.tolist()
X_test_new_seq = tokenizer.texts_to_sequences(X_test_new)
X_test_new_padded = pad_sequences(X_test_new_seq, padding='post', maxlen=8)

# Then we predict using the trained model
predictions_new = model.predict(X_test_new_padded)
predicted_classes_new_indices = np.argmax(predictions_new, axis=1)
predicted_class_labels_new = label_encoder.inverse_transform(predicted_classes_new_indices)
prediction_new_df = pd.DataFrame({
    'Original_X_test': X_test_new,
    'Predicted_Category': predicted_class_labels_new
})
data_class  = DataClass(use_prediction= True,use_enhanced=True,custom=True)

prediction_new_df['true'] = data_class.data.loc[data_class.enhanced_index]['groundtruth'].values
prediction_new_df
data_class.data.loc[data_class.enhanced_index,'firstname_lower_lstm'] = predicted_class_labels_new

In [ ]:
data_class.data.loc[data_class.enhanced_index,'firstname_lower_lstm']

In [ ]:
from fuzzywuzzy import process
choices = data_model['Y'].unique().tolist()
all_pred = []
for x in X_test_new:
    pred = process.extract(x,choices= choices,limit=1)[0][0]
    all_pred.append(pred)

In [51]:
%load_ext autoreload 
%autoreload 2 
from modelling import *
data_class  = DataClass(use_prediction= True,use_enhanced="lstm",custom=True)
data_class.create_dataset()
modelling_data = DataModel(data = data_class.data_model,
                           use_enhanced= data_class.use_enhanced,
                           custom_test_index=data_class.enhanced_index,
                           max_len=200,
                           custom=True)
modelling_data.create_padding()
modelling_data.test
data_class.data_model.loc[data_class.enhanced_index]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
modifying features
['link', 'employer', 'occupation', 'name_sex', 'firstname_lower']
['link', 'employer', 'occupation', 'name_sex', 'firstname_lower_lstm']
Shape of training tensor:  (205, 200)
Shape of testing tensor:  (27, 200)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,message,label
18,ch prenom femme ang,1
24,patron prenom femme hermance,1
25,ep prenom femme marguerite,0
34,ep prenom femme silvie,0
65,prenom femme angeline,1
69,metages prenom homme felix,1
82,prenom homme george,1
104,patron prenom femme gilberte,1
121,fr re labrye manoeuvre prenom homme francisque,1
126,fils prenom homme magloire,1


In [52]:
model_socface = Model(data = modelling_data,model_name= "mlp")
model_socface.fit()
model_socface.predict()
model_socface.compute_results()
model_socface.model.table_results

Epoch 1/200
6/6 - 3s - loss: 1.0968 - accuracy: 0.4573 - val_loss: 0.8198 - val_accuracy: 0.3659 - 3s/epoch - 454ms/step
Epoch 2/200
6/6 - 1s - loss: 0.7451 - accuracy: 0.5549 - val_loss: 0.6541 - val_accuracy: 0.6341 - 1s/epoch - 192ms/step
Epoch 3/200
6/6 - 2s - loss: 0.7104 - accuracy: 0.4939 - val_loss: 0.7058 - val_accuracy: 0.3659 - 2s/epoch - 262ms/step
Epoch 4/200
6/6 - 1s - loss: 0.6988 - accuracy: 0.5793 - val_loss: 0.6051 - val_accuracy: 0.6341 - 1s/epoch - 218ms/step
Epoch 5/200
6/6 - 1s - loss: 0.6119 - accuracy: 0.7012 - val_loss: 0.5352 - val_accuracy: 0.9268 - 1s/epoch - 226ms/step
Epoch 6/200
6/6 - 1s - loss: 0.4932 - accuracy: 0.8415 - val_loss: 0.5001 - val_accuracy: 0.8049 - 1s/epoch - 218ms/step
Epoch 7/200
6/6 - 1s - loss: 0.3240 - accuracy: 0.9024 - val_loss: 0.2319 - val_accuracy: 0.9268 - 1s/epoch - 213ms/step
Epoch 8/200
6/6 - 1s - loss: 0.1164 - accuracy: 0.9756 - val_loss: 0.1606 - val_accuracy: 0.9268 - 1s/epoch - 212ms/step
Epoch 9/200
6/6 - 1s - loss: 0.0

NameError: name 'y_pred' is not defined